In [1]:
%load_ext autoreload
%autoreload 2
import pynamod
import numpy as np
import pandas as pd
import MDAnalysis as mda
import nglview as nv
import itertools
from numba import jit
import copy

/home/v_vasilev/.conda/envs/pynamod_new/lib/python3.12/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/v_vasilev/.conda/envs/pynamod_new/lib/python3.12/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


/home/v_vasilev/.conda/envs/pynamod_new/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
nucl = pynamod.DNA_structure_from_atomic(pdb_id='3LZ0',leading_strands=['I'])

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


In [4]:
nucl.analyze_DNA()

In [5]:
nucl.analyze_protein()

[[ 35.84986863   7.92918331 -13.32614108]
 [ 65.49836972  -8.3934404    5.45107891]
 [ 68.98541382  26.16405216   5.61804262]
 [ 44.07350291  18.45744568   2.8796968 ]
 [ 56.30823907  21.70385317  14.94109553]
 [ 50.75735465 -11.08906742 -17.9116746 ]
 [ 24.82907001  -8.30798765 -19.49813012]
 [ 27.85616197  -4.68384858   0.42389547]
 [ 36.27321348  15.65324928  14.19243089]
 [ 53.47182764 -11.5064807    1.80331971]
 [ 41.56138031 -24.14079929 -21.20831111]
 [ 24.52511335   1.38818497 -23.10619295]
 [  9.79660652  -7.62828349 -26.46685393]
 [ 68.59374679 -25.68642732  -6.6941959 ]
 [ 44.22006002  -4.70053165  24.6497283 ]
 [ 39.00993523  19.07476241 -23.29676598]
 [ 15.62089671  -9.72754026 -11.9414201 ]
 [ 18.25076517  10.87992217  15.765493  ]
 [ 68.31892836   3.18735046  -5.05603398]
 [ 34.15271953   9.96775972 -26.14905894]
 [ 63.39376645   9.2264964   19.8040333 ]
 [ 39.98135797 -16.64437493  -8.4147612 ]
 [ 21.52407611   9.23316819 -16.08613738]
 [ 48.38443329   6.19047261   9.12

In [6]:
nucl.move_to_coord_center()

In [10]:
nucl.proteins[0].cg_charges

array([-2.2830997 ,  1.96150005,  3.69659982, -1.04089964,  0.16680005,
        1.97720009,  2.41500007, -0.56679965, -0.56379986, -0.28509979,
        0.05439994,  3.59470043,  4.94570018,  2.74509995,  2.91720004,
        1.85349995,  0.51029997,  3.69420006,  0.09920008,  1.28300027,
        5.24560032, -1.88219971,  0.04760018, -3.10020007,  0.79389994,
        4.03630027,  0.62680012,  2.42650008,  3.02960013,  1.93949999,
        2.67380029,  2.09210018,  2.42040014,  0.7591    , -1.20340008,
        0.46270008,  2.87920015, -3.03979971,  2.55520017,  1.14940013,
        1.48839997,  1.79869972,  3.28540041,  4.2030001 ,  1.8915    ,
       -0.10260005,  4.50469998,  2.01140011,  0.84300017, -3.01029977])

In [8]:
nucl.view_structure()

NGLWidget()

In [447]:
nucl.get_DataFrame()

,resid1,segid1,restype1,restype2,segid2,resid2,Shear,Stretch,Stagger,Buckle,Prop-Tw,Opening,Shift,Slide,Rise,Tilt,Roll,Twist
0,-72,I,A,T,J,72,0.097097,0.121459,1.045064,14.008287,-8.186396,-18.747150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-71,I,T,A,J,71,0.616557,-0.423763,0.366543,-3.207604,-21.722571,-8.927834,-0.103346,-1.031617,4.009800,5.432016,-2.159719,37.183770
2,-70,I,C,G,J,70,0.833555,0.265378,1.064210,-28.183521,-17.827698,10.565053,1.007070,-0.137341,3.687516,-0.260508,-0.061271,41.364948
3,-69,I,A,T,J,69,0.628470,-0.224859,-0.341789,-14.207446,-18.915283,-9.267536,-0.740422,0.309669,2.985010,7.616249,11.300466,28.885169
4,-68,I,G,C,J,68,-0.782726,-0.361822,0.376319,-0.182252,-13.276127,9.313960,1.459504,-0.116359,3.047685,-5.397753,-4.828530,32.891446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,68,I,T,A,J,-68,-0.792183,-0.256568,-0.150800,-17.477464,-8.907526,-2.940552,0.317223,0.315937,3.053923,0.281955,-1.353624,32.618715
141,69,I,C,G,J,-69,0.400506,0.254092,0.207221,-13.002930,-22.554792,13.191512,0.557327,1.586954,3.321717,1.464002,-5.845326,45.378707
142,70,I,G,C,J,-70,-0.297763,-0.296159,0.543946,-0.396752,-3.938952,-2.735247,0.108807,0.536010,2.799119,-5.863605,9.525713,27.927832
143,71,I,A,T,J,-71,-0.363076,0.392318,-0.576545,-14.754146,-19.957727,-11.636812,-0.600595,-0.435970,3.632289,4.446861,5.686760,41.164965


In [486]:
dna_gen = pynamod.DNA_structure_generated('atcg'*15)

In [482]:
nucl_temp = pynamod.DNA_structure_from_atomic(pdb_id='3LZ0',leading_strands=['I'])
nucl_temp.analyze_DNA()
nucl_temp.analyze_protein()

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


In [489]:
nucl.append_structures((dna_gen,nucl_temp))

In [331]:
nucl.view_structure()

NGLWidget()

In [488]:
nucl.copy()

AttributeError: 'DNA_structure_from_atomic' object has no attribute 'copy'

In [41]:
df = nucl.get_DataFrame()

In [510]:
u = mda.Universe(file.name)

In [480]:
np.sum(u.select_atoms('protein').charges)

68.00000557862222

In [511]:
del file

In [459]:
import subprocess
import tempfile

In [535]:
file = tempfile.NamedTemporaryFile(suffix='.pqr')

In [536]:
file.name

'/tmp/tmpjqr4atqx.pqr'

In [537]:
process = subprocess.run(['pdb2pqr', '--ff=AMBER','--log-level=CRITICAL', '3lz0.pdb', file.name])

In [ ]:
u.atoms.write(file.name)

In [542]:
mda.Universe(file.name).atoms.charges

array([-0.20200001,  0.1       ,  0.52600002, ...,  0.0718    ,
        0.0718    ,  0.43959999])

In [498]:
u.select_atoms('not type H')

<AtomGroup with 11902 atoms>

In [499]:
nucl.u

<Universe with 11908 atoms>

In [516]:
u.atoms

<AtomGroup with 21461 atoms>

In [517]:
from os import path

In [521]:
import os

In [522]:
os.getcwd()

'/home/_shared/_projects/2024_pynamod_update/package/PyNAMod'